# **First assignment: Informed and adversarial search strategies**

## **Connect 4 Game**

Implementação de uma classe que representa o jogo Connect 4.

### **Implementação:**

In [ ]:
class FourGame():
    # Parameters | self: Class FourGame instance | columns: Integer number of columns for the game | lines: Integer number of lines for the game
    def __init__(self, columns, lines):          # Class constructor (there is no need to declare the attributes since the self.attribute does it for us)
        self.state = [['-' for _ in range(columns)] for _ in range(lines)]
        self.columns = columns
        self.lines = lines
        self.plays = 0
        self.toPlay = 'O'
        self.result = None


    # Parameters | self: Class FourGame instance
    def __str__(self):                          # Returns the string who graphically represents the matrix of the game
        str = "=============================\n"
        for line in self.state:                 # Iterate through the columns
            for sign in line:                   # Iterate through the lines
                str += '| ' + sign + ' '
            str += "|\n"
        str += "=============================\n"
        str += "  1   2   3   4   5   6   7  \n"
        return str


    # Parameters | self: Class FourGame instance | column: Integer number of column to play | Character symbol ('X', 'O')
    def __insertSymbol(self, column, symbol):   # Private method to insert a symbol into the array
        for i in range(self.lines-1, -1, -1):
            if self.state[i][column] == '-':
                self.state[i][column] = symbol
                if symbol == 'X':
                    self.toPlay = 'O'
                else:
                    self.toPlay = 'X'
                return i
        return -1                               # Returns in case the column is full

    def __checkRepetitions(self, line, column, lineCount, columnCount, symbol):
        count = 0
        lineI = line
        columnI = column
        for _ in range(2):

            while lineI <= 5 and lineI >= 0 and columnI <= 6 and columnI >= 0 and self.state[lineI][columnI] == symbol:
                count += 1
                columnI += columnCount
                lineI += lineCount
                if count == 4: 
                    self.result = symbol
                    return True
 
            columnCount *= -1
            lineCount *= -1
            columnI = column + columnCount
            lineI = line + lineCount

        return False
    
    def __checkWin(self, column, line, symbol):

        # Check for win vertically
        if self.__checkRepetitions(line, column, -1, 0, symbol):
            return True
            
        # Check for win horizontallys
        if self.__checkRepetitions(line, column, 0, 1, symbol):
            return True
        
        # Check for win main diagonal
        if self.__checkRepetitions(line, column, -1, 1, symbol):
            return True

        # Check for win inverse diagonal
        if self.__checkRepetitions(line, column, 1, 1, symbol):
            return True
        
        return False
        
    
    # Parameters | self: Class FourGame instance | column: Integer number of column to play | Character symbol ('X', 'O')
    # Return: string or false
    def makeMove(self, column, symbol):
        
        line = self.__insertSymbol(column - 1, symbol)
        if line == -1: return -1, ''  # Invalid move, the column is full

        self.plays += 1
        
        # Verify if it should end the game (in case if someone wins or the board is full)
        if self.__checkWin(column - 1, line, symbol): 
            return 2, symbol
        elif (self.plays == self.columns*self.lines): 
            return 1, ''
        return 0, ''

    def gameOver(self):
        return self.result is not None or self.plays == self.columns * self.lines
    
    def gameDraw(self):
        return self.plays == self.columns * self.lines
    
    def getLegalMoves(self):
        legalMoves = []
        for i in range(7):
            if self.state[0][i] == '-':
                legalMoves.append(i)

        return legalMoves

## **Estruturas de Dados implementadas**

### **Node**

#### **Atributos**

&emsp;Move: Coordenadas do Movimento feito a ser representado pelo nó. Vetor de duas dimensões com a posição da coluna e linha.

&emsp;State: Estado atual do tabuleiro do jogo. Matriz de caracteres que representam os simbolos do jogo.

&emsp;Parent: Nó pai do nó representado. Referência para um nó.

&emsp;Children: Lista dos nós filhos do nó. Lista de referências para nós.

&emsp;pathCost: Custo do caminho + heurística para passar para o nó (o custo neste contexto não é relevante e não é considerado)

&emsp;N: A preencher

&emsp;Q: A preencher

<br>

#### **Métodos**

&emsp;setPathCost: Atribui um valor ao atributo pathCost

&emsp;getPathCost: Retorna o valor do atributo pathCost

&emsp;setChildren: Atribui um valor ao atributo children

<br>

### **Implementação:**

In [ ]:
class Node():

    def __init__(self, move, state, parent):
        self.move = move
        self.state = state
        self.parent = parent
        self.children = []
        self.N = 0
        self.Q = 0
    
    def setPathCost(self, cost):
        self.pathCost = cost

    def getPathCost(self):
        return self.pathCost
    
    def setChildren(self, children):
        self.children = children

### **Vector**

#### **Atributos**

&emsp;X: Valor X do vetor. Inteiro

&emsp;Y: Valor Y do vetor. Inteiro

<br>

#### **Métodos**

&emsp;setX: Atribui um valor ao atributo X

&emsp;setY: Atribui um valor ao atributo Y

&emsp;getX: Retorna o valor do atributo X

&emsp;getY: Retorna o valor do atributo Y

<br>

### **Implementação:**

In [ ]:
class Vector():

    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def setX(self, x):
        self.x = x

    def setY(self, y):
        self.y = y
    
    def getX(self):
        return self.x

    def getY(self):
        return self.y

### **Queue**

#### **Atributos**

&emsp;Stack: Lista para armazenar os valores da fila.

&emsp;Size: Tamanho da fila.

<br>

#### **Métodos**

&emsp;isEmpty: Verifica se a lista está ou não vazia.

&emsp;pop: Remove e retorna o elemento do topo da fila.

&emsp;top: Apenas retorna o elemento do topo da fila.

&emsp;add: Adiciona um elemento no topo da fila.

<br>

### **Implementação:**

In [ ]:
class Queue():

    def __init__(self):
        self.stack = []
        self.size = 0

    def isEmpty(self):
        if self.size ==0:
            return True
        return False

    def pop(self):
        if self.size != 0:
            self.size = self.size - 1
            return self.stack.pop(self.size)
        else:
            return None

    def top(self):
        if self.size != 0:
            return self.stack.pop(self.size-1)
        else:
            return None
        
    def add(self, value):
        self.stack.append(value)
        self.size = self.size + 1

## **Heurística**

A heurística apresentada lida bem com o problema, mas por vezes parece não favorecer vitórias mais rápidas. No entanto decidimos ainda experimentar outra heurística de modo a comparar.

A heurística dada no enunciado tem como base a expressão algébrica seguinte: ValorPeçasIA - ValorPeçasDoAdversário

Este valor de peças é dado pela soma das combinações em linha dos jogadores em todo o tabuleiro de 4 em 4 slots e cada linha de 4 tem os seguintes valores:

&emsp;Em caso de tokens iguais apenas: 1 em linha = 1, 2 em linha = 10, 3 em linha = 50, 4 em linha = 512 .

&emsp;Em caso de haver tokens de ambos os lados na mesma linha de 4 que basicamente não podem resultar em vitória é dado o valor 0.

Experimentamos ainda mudar os valores acima para 0.1 0.3 e 0.9 3.2 de modo a experimentar baseado num documento de um trabalho realizado por um aluno de outra faculdade, mas não era tão eficiente dado que permitia o empate com o MinMax (pelo menos para a profundidade de 3).

Outra heurística possível é valorizar cada slot do tabuleiro dado mais valor aos do centro, como mostrado no seguinte esquema:

<div style="text-align: center;">
    <img src="esquemas/heuristica.png" alt="Esquema MCTS" width="320" class="center"/>
</div>

Testando usar esta heurística sozinha, para estado iniciais ela performa muito bem, mas em estado mais avançados comete erros e não é suficiente.

Mas se a usarmos junto à heurística anterior então temos uma boa adição a ela. Isto no caso do MinMax visto que no A* não é propriamente impactante visto que ele perde por não se defender (não ter em conta o adversário).

<br>

### **Implementação:**

In [ ]:
# Regras de valoração de combinações do segmento (segmentos de 4 slots)
# Parâmetros:

# segment | type: list de caracteres de tamanho 4   -> representa um segmento de 4 slots com os símbolos de cada slot
# IaSymbol | type: string                           -> representa o símbolo que IA está a usar para jogar

# returns: type: int                                -> Retorna a valoração correta para a combinação de número de símbolos X e O encontrados no segmento segundo as regras dadas para calculo da heurística

def heuristicVal(segment, IaSymbol):
    
    # Define qual símbolo é usado pelo humano com base no dado para a IA
    if IaSymbol == 'X':
        HumanSymbol = 'O'
    else:
        HumanSymbol = 'X'

    # Conta o número
    IA_count = segment.count(IaSymbol)
    Human_count = segment.count(HumanSymbol)

    if Human_count == 4 and IA_count == 0:
        return -512                                                     # Valor para 4 Símbolos do humano e 0 Símbolos  da IA
    elif Human_count == 3 and IA_count == 0:
        return -50                                                      # Valor para 3 Símbolos do humano e 0 Símbolos  da IA
    elif Human_count == 2 and IA_count == 0:
        return -10                                                      # Valor para 2 Símbolos do humano e 0 Símbolos  da IA
    elif Human_count == 1 and IA_count == 0:
        return -1                                                       # Valor para 1 Símbolos do humano e 0 Símbolos  da IA
    elif Human_count == 0 and IA_count == 0:
        return 0                                                        # Sem O ou X
    elif IA_count == 1 and Human_count == 0:
        return 1                                                        # Valor para 1  Símbolos da IA e 0 Símbolos do humano
    elif IA_count == 2 and Human_count == 0:
        return 10                                                       # Valor para 2  Símbolos da IA e 0 Símbolos do humano
    elif IA_count == 3 and Human_count == 0:
        return 50                                                       # Valor para 3  Símbolos da IA e 0 Símbolos do humano
    elif IA_count == 4 and Human_count == 0:
        return 512                                                      # Valor para 4  Símbolos da IA e 0 Símbolos do humano
    else:
        return 0                                                        #Combinação de O e X | nenhum padrão encontrado


# Cálculo da heurística através da geração dos segmentos e soma dos seus valores 
# Parâmetros: 

# boardState | matrix (list of lists): matrix de caracteres             -> representa o estado atual do tabuleiro do jogo
# IaSymbol | type: string                                               -> representa o símbolo que AI está a usar para jogar

# returns: type: int                                                    -> Retorna a heurística do boardState dado

def heuristicCalculate(boardState, IaSymbol, coords):

    ValueGrade = [[3,4,5,7,5,4,3],[4,6,8,10,8,6,4],[5,8,11,13,11,8,5],[5,8,11,13,11,8,5],[4,6,8,10,8,6,4],[3,4,5,7,5,4,3]]

    heuristic = 0

    if coords:
        heuristic =  ValueGrade[coords.y][coords.x]

    # Segmentos horizontais
    for row in boardState:
        for i in range(len(row) - 3):
            segment = row[i:i+4]                                        # Segmento a ser valorado
            heuristic += heuristicVal(segment, IaSymbol)

    # Segmentos verticais
    for j in range(len(boardState[0])):
        for i in range(len(boardState) - 3):
            segment = [boardState[i+k][j] for k in range(4)]            # Segmento a ser valorado
            heuristic += heuristicVal(segment, IaSymbol)

    # Diagonal segments (superior-esquerdo to inferior-direito)
    for i in range(len(boardState) - 3):
        for j in range(len(boardState[0]) - 3):
            segment = [boardState[i+k][j+k] for k in range(4)]          # Segmento a ser valorado
            heuristic += heuristicVal(segment, IaSymbol)

    # Segmentos das Diagonais (superior-direito até inferior-esquerdo)
    for i in range(len(boardState) - 3):
        for j in range(3, len(boardState[0])):
            segment = [boardState[i+k][j-k] for k in range(4)]          # Segmento a ser valorado
            heuristic += heuristicVal(segment, IaSymbol)

    # Bónus de movimento
    move_bonus = 16 if IaSymbol == 'X' else -16                         # Define o bónus dependendo de quem joga
    heuristic += move_bonus                                             # Adiciona o bónus

    return heuristic

## **Algoritmo A\* (Algoritmo não advesarial)**

### **Implementação:**

In [1]:
from dataStructs.myQueue import Queue
from dataStructs.node import Node
from dataStructs.vector import Vector
from searchAlgos.heuristic import heuristicCalculate

# Máximo entre dois inteiros
# Parâmetros: 

# a, b | type: int                                       -> Inteiros a ser comparados

# returns: type: int                                     -> Retorna o máximo entre a e b

def max(a, b):
    if a > b:
        return a
    else:
        return b

# Classe que representa o algoritmo Astar                                                                                                                                                  
# Atributos:                                                                                                                                                                                
       
# frontier | type: Queue instance                        -> Representa a fronteira ou seja os próximos caminhos a partir do estado dado
# symbol | type: string                                  -> Representa o símbolo a ser jogado pela IA
# monotony | type: int/float                             -> Representa o valor da heurística (custo do caminho é irrelevante neste jogo) do ultimo estado escolhido

# Métodos:

# __bestMove | privado                                   -> Retorna a coluna do move do melhor nó ou seja o nó com heurística mais alta no caso
# __setFrontier | privado                                -> Define o atributo frontier com os nós filhos (neste caso não adiciona os nós anteriormente analizados pois não podemos voltar para trás neste jogo)
# play | publico                                         -> Retorna a jogada a fazer pela IA

class Astar():

    # Construtor da classe
    # Parâmetros: 
    
    # self | type: int                                    -> Referência ao objeto Astar usado 
    # symbol | type: string                               -> Símbolo jogado pela IA a ser guardado como argumento da classe
    
    # A monotonia é ainda inicializada com o -inf para que possa ser ignorada na primeira jogada da IA

    def __init__(self, symbol):
        self.frontier = Queue()
        self.symbol = symbol
        self.monotony = float('-inf')
    
    # Best Move
    # Parâmetros: 
    
    # self | type: int                                                  -> Referência ao objeto Astar usado 
    
    # return: type: int                                                 -> Retorna a coluna do move do melhor nó ou seja o nó com heurística mais alta no caso

    def __bestMove(self):

        if self.frontier.isEmpty():                                     # Verifica se exitem jogadas possíveis (nós filhos)
            return False

        bestMoveNode = self.frontier.pop()                              # Escolhe o melhor nó de entre os nós filhos (ou seja aquele com melhor heurística)
        while ((newNode := self.frontier.pop()) != None):
            if newNode.pathCost > bestMoveNode.pathCost:
                bestMoveNode = newNode
        
        self.monotony = bestMoveNode.pathCost                           # Atualiza a monotonia para a proxima jogada da IA

        return bestMoveNode.move.getX()
    
    # setFrontier
    # Parâmetros: 
    
    # self | type: int                                                  -> Referência ao objeto Astar usado 
    # actualState | type: matrix (list of lists): matrix de caracteres  -> Representa o estado atual do tabuleiro do jogo

    # Define o atributo frontier com os nós filhos (neste caso não adiciona os nós anteriormente analizados pois não podemos voltar para trás neste jogo)
    def __setFrontier(self, actualState):
        for column in range(0,7):
            for line in range(5,-1,-1):
                if actualState[line][column] == '-':

                    newState = list(map(list, actualState))             # Cria uma cópia do tabuleiro numa referência de memória diferente
                    newState[line][column] = self.symbol                # Atualiza o tabuleiro com a jogada representada por o nó a ser calculado

                    node = Node(Vector(column, line), newState, None)   # Cria o nó com o move e o estado do jogo que representa

                    # Na linha abaixo deveria ser somado o custo do caminho ao heuristicCalculate mas dado que neste jogo o custo é irrelevante não foi feito.
                    node.setPathCost(max(self.monotony, heuristicCalculate(node.state, self.symbol, node.move))) # Calcula e guarda a heurística do nó especifico (o custo do caminho é irrelevante neste jogo)

                    self.frontier.add(node)                             # Adiciona o nó à fronteira
                    break

    # play
    # Parâmetros: 
    
    # self | type: int                                                  -> Referência ao objeto Astar usado 
    # game | type: matrix (list of lists): matrix de caracteres         -> Representa o estado atual do tabuleiro do jogo
    
    # return | type: int                                                -> Retorna a jogada a fazer pela IA

    def play(self, game, _):
        self.__setFrontier(game.state)                                  # Define a fronteira
        
        return self.__bestMove()                                        # Verifica e retorna o melhor dos nós da fronteira


### **Dificuldades sentidas e pontos abordados durante a implementação do algoritmo A\***

Numa primeira fase quando começamos a implementar o algoritmo e percebemos que ao contrário da implementação normal do A* esta não iria necessitar voltar para trás visto que no jogo não existe a possibilidade de remover peças do tabuleiro. Outra conclusão que tirámos era que o custo do caminho neste jogo não seria necessário visto que é o mesmo para qualquer coluna jogada de certa forma não existe um custo associado. Isso levou-nos à conclusão que estariamos a implementar um Greedy em vez de um A*.

A maior dificuldade neste algoritmo foi enquanto não percebemos que o comportamento de ignorar o outro jogador se dava devido à sua natureza não-advesarial.

<br>

### **Problemas do algoritmo no contexto apresentado**

Durante a implementação e testes no A* notamos principalmente dois problemas principais:

O facto do A* ser *não adversarial*, faz com que não reconheça as jogadas do adversário, perdendo sem se defender. Na maioria dos casos, existe uma grande facilidade de vitória por parte do jogador humano especialmente se este jogar em  primeiro lugar.

O segundo não seria um problema, mas o facto de não existir um custo associado a jogar em determinada coluna visto que é o mesmo jogar em qualquer que seja pelo menos em termos de custo (a heurística é diferente no entando esta é baseada no resultado final), estamos basicamente a implementar um Greedy. 

Dado que implementar a combinação do custo somado da heurística como custo total é o mesmo que considerar apenas a heurística tendo isto em conta, consideramos apenas a heurística, de certa forma implementando um greedy a única diferença é que garantimos que a função é monótona (isto não está presente no greedy).

Foi deixado um comentário na parte do código onde deveria ter sido feita a soma do custo do caminho para assim ter mesmo um A*.

<br>

## **Algoritmo MinMax (Adversarial)**

### **Implementação:**

In [ ]:
from dataStructs.vector import Vector
from dataStructs.node import Node
from searchAlgos.heuristic import heuristicCalculate

# Atributos:                                                                                                                                                                                
     
# MaxSymbol | type: character       -> Representa o simbolo usado nos máximos (Simbolo da IA)
# MinSymbol | type: character       -> Representa o símbolo usado nos mínimos

# Métodos:
#
#  __gameAlreadyWon                 ->  Retorna a coluna do move do melhor nó ou seja o nó com heurística mais alta no caso
#  __minimax                        ->  Função recursiva para determinar o máximo e minimo dos nós alternadamente (implementação do minMax em si)
#  __getChildren                    ->  Define os nós filhos do nó a ser analisado e dá calcula a heurística deles e o estado que representam
#  play                             ->  Retorna a jogada a fazer pela IA
#
class MinMax():

    # Construtor da classe
    # Parâmetros: 
    
    # self      | type: int         -> Referência ao objeto MinMax usado
    # MaxSymbol | type: character   -> Representa o simbolo usado nos máximos (Simbolo da IA)
    # MinSymbol | type: character   -> Representa o símbolo usado nos mínimos
    
    def __init__(self, MaxSymbol, MinSymbol):
        self.MaxSymbol = MaxSymbol
        self.MinSymbol = MinSymbol
    
    # gameAlreadyWon
    # Parâmetros:
    
    # self | type: int                                                      -> Referência ao objeto MinMax usado
    # gameState | type: matrix (list of lists): matrix de caracteres        -> Representa o estado do jogo neste ponto
    
    # return: type: int                                                     -> Retorna a heurística em caso de vitória, empate ou derrota ou None

    def __gameAlreadyWon(self, gameState):
        heuristic = heuristicCalculate(gameState, self.MaxSymbol, None)     # Calcula a heurística do estado neste ponto
        if heuristic >= 512 or heuristic <= -512 or heuristic == 0:         # Verifica se é vitória, empate ou derrota
            return heuristic
        return None

    # minimax
    # Parâmetros: 
    
    # self | type: int                                                      -> Referência ao objeto MinMax usado
    # children | type: list of nodes                                        -> Lista com os nós filhos do nó "expandido"
    # depth | type: int                                                     -> Altura restante a analisar
    # maximizingPlayer | type: bool                                         -> Se é ou não uma altura de comparar máximos ou não (se não for são mínimos)
    # actualGame | type: matrix (list of lists): matrix de caracteres       -> Representa o estado do jogo neste ponto
    
    # return: type: int                                                     ->  Retorna o valor máximo ou minimo dependendo da fase e o melhor nó


    def __minimax(self, children, depth, maximizingPlayer, actualGame):

        if depth == 0:                                                       # Verifica se a já verificamos a uma altura suficiente (defenida posteriormente)
            return 0, None
 
        # Primeiro expande até à profundidade pretendida e posteriormente ao voltar para trás escolhe o nó maximo ou minimo dependendo do "turno" (forma alternada começando sempre no Max nesta implementação)
        if maximizingPlayer:
            heuristicVal = self.__gameAlreadyWon(actualGame)                # Verifica se houve ou não o fim do jogo e retorna uma heurística caso aconteça
            if heuristicVal and (heuristicVal >= 512 or heuristicVal == 0):
                return heuristicVal, None                                   # Retornando tal heurística como valor máximo em caso de vitória ou empate
 
            max_eval = float('-inf')                                        # Inicializa o valor máximo como -inf para a primeira comparação
            best_move = None                                                # Inicializa o bestMove
            for child in children:                                          # Percorre os nós filhos do nó atual a ser analizado

                eval, bestChild = self.__minimax(self.__getChildren(child.state, self.MinSymbol), depth - 1, False, child.state) # Para cada nó filho aplica o minmax mas agora para a busca de minimos

                if bestChild == None:                                       # Caso o nó maximo dos nós seja vitória ou empate o eval recebe o valor da heurística do nó filho a ser analizado
                    eval = child.getPathCost()

                if eval > max_eval:                                         # Verifica se o nó é maximo comparado aos já analizado
                    max_eval = eval
                    best_move = child

            return max_eval, best_move                                      # Retorna o nó maximo e o seu valor
        else:
            heuristicVal = self.__gameAlreadyWon(actualGame)                # Verifica se houve ou não o fim do jogo e retorna uma heurística caso aconteça
            if heuristicVal and (heuristicVal <= -512 or heuristicVal == 0): 
                return heuristicVal, None                                   # Retornando tal heurística como valor mínimo em caso de vitória ou empate

            min_eval = float('inf')                                         # Inicializa o valor mínimo como inf para a primeira comparação
            best_move = None                                                # Inicializa o bestMove
            for child in children:                                          # Percorre os nós filhos do nó atual a ser analizado

                eval, bestChild = self.__minimax(self.__getChildren(child.state, self.MaxSymbol), depth - 1, True, child.state) # Para cada nó filho aplica o minmax mas agora para a busca de máximos
                
                if bestChild == None:                                       # Caso o nó mínimo dos nós seja vitória ou empate o eval recebe o valor da heurística do nó filho a ser analizado
                    eval = child.getPathCost()
                
                if eval < min_eval:                                         # Verifica se o nó é maximo comparado aos já analizado
                    min_eval = eval
                    best_move = child

            return min_eval, best_move                                      # Retorna o nó mínimo e o seu valor

    # getChildren
    # Parâmetros: 
    
    # self | type: int                                                      -> Referência ao objeto MinMax usado
    # game | type: matrix (list of lists): matrix de caracteres             -> Representa o estado do jogo neste ponto
    # symbolToPlay | type: character                                        -> Simbolo a jogar no momento
    
    # return: type: list of nodes                                           -> Retorna os nós filhos do estado passado

    def __getChildren(self, game, symbolToPlay):
        children = []
        for column in range(0,7):
            for line in range(5, -1, -1):
                if game[line][column] == '-':                                               # Procura pela linha disponível na coluna caso exista
                    
                    newGame = list(map(list, game))                                         # Copia o estado do tabuleiro para uma referência de memória diferente
                    newGame[line][column] = symbolToPlay                                    # Atualiza esse estado com o simbolo no move a jogar
                    node = Node(Vector(column, line), newGame, None)                        # Cria um nó para esse move e estado
                    
                    node.setPathCost(heuristicCalculate(game, self.MaxSymbol, node.move))   # Calcula a heurística para esse estado
                    children.append(node)                                                   # adiciona esse nó á lista dos filhos
                    break
        return children

    # play
    # Parâmetros: 
    
    # self | type: int                                                      -> Referência ao objeto MinMax usado
    # game | type: matrix (list of lists): matrix de caracteres             -> Representa o estado do jogo neste ponto
    
    # return: type: int                                                     -> Retorna a coluna da melhor jogada

    def play(self, game, _):
        newGame = list(map(list, game.state))                               # Copia o estado do jogo para outro endereço de memória
        frontier = self.__getChildren(newGame, self.MaxSymbol)              # Busca os nós filhos do estado atual
    
        _, melhor = self.__minimax(frontier, 3, True, newGame)              # Aplica o minmax começando pelo máximo

        return melhor.move.getX()                                           # Retorna a coluna da melhor jogada

### **Análise do MiniMax**
Este algoritmo funciona recursivamente, sobre as possíveis jogadas, apartir de uma dada posição do tabuleiro de jogo.

Utiliza o cálculo da heurística para avaliar se o nó recebido através da função setFrontier que possui as jogadas válidas numa posição, e escolhe a jogada que neste caso maximiza a sua heurística.

O funcionamento descrito acima, refere-se a um jogo entre o algoritmo e o humano, porém este também é capaz de jogar contra ele próprio, utilizando recursivamente a chamada da função com um argumento diferente, desta vez para minimizar a sua heurística, alternadamente minimizando e maximizando a heurística do jogador em questão.

<br>

#### **Esquema:**


<div style="text-align: center;">
    <img src="esquemas/minimax.png" alt="Esquema MCTS" width="1200" class="center"/>
</div>

<br>

### **Melhorias em relação ao A\* e problemas:**

Decidimos implementar o MinMax para testar realmente a heurística usando um algoritmo adversarial que é ideal para o jogo em questão.

Comparando com o A* a diferença é notória pois o MinMax considera as jogadas do adversário, bloqueando-as e jogando em função de evitar a derrota e não apenas de ganhar.

Reparámos que por vezes o algortimo não escolhe o caminho de vitória mais rápido optando por fazer mais jogadas mas isso deve-se principalmente à heurística apresentada.

Em termos de performance depende da profundidade a ser analisada pretendida, uma profundidade de 3 pareceu-nos ideal e uma boa troca entre performance e eficiência, sendo praticamente impossível de ganhar (nós nunca o conseguimos fazer), obviamente profundidades maiores tornavam o algoritmo ligeiramente mais lento mas tornava-o mais díficil de vencer teoricamente.

<br>

## **Algoritmo Monte Carlo Tree Search (Adversarial)**

### **Implementação:**

In [ ]:
from dataStructs.node import Node
from copy import deepcopy
import random
import time

class MCTS():
    def __init__(self, iaSymbol, game):
        self.root = Node(None, None, None)                                                      # Inicia a raiz R com valores None
        self.rootState = deepcopy(game)                                                         # Define o estado do jogo na raiz como uma cópia do objeto game da classe FourGame
        self.symbol = iaSymbol                                                                  # Define o símbolo do MCTS como o iaSymbol (argumento passado na interface)
        self.playerSymbol = (iaSymbol == 'X' and 'O') or (iaSymbol == 'O' and 'X')              # Define o símbolo do jogador como o símbolo contrário do MCTS
        self.iaSymbol = iaSymbol                                                                
        self.numRollouts = 0                                                                    # Inicializa o número de Rollouts a 0 (dado estatístico)
        self.runTime = 0                                                                        # Inicializa o tempo decorrido a 0 (dado estatístico)

    def __rotateSymbol(self):                                                                   # Uma função para definir a rotação dos símbolos
        if self.symbol == 'X':
            self.symbol = 'O'
            return 'X'
        else:
            self.symbol = 'X'
            return 'O'

    def __selection(self):
        node = self.root
        state = deepcopy(self.rootState)

        while len(node.children) != 0:
            children = node.children.values()
            max_value = max(children, key=lambda n: n.value()).value()
            maxChildren = [n for n in children if n.value() == max_value]

            node = random.choice(maxChildren)
            state.makeMove(node.move+1, self.__rotateSymbol())

            if node.N == 0:
                return node, state

        if self.__expand(node, state):
            node = random.choice(list(node.children.values()))
            state.makeMove(node.move+1, self.__rotateSymbol())
            
        return node, state
    
    def __expand(self, parent, state):

        if state.gameOver():
            return False
        
        children = [Node(move, None, parent) for move in state.getLegalMoves()]
        parent.setChildren(children)

        return True

    def rollOut(self, state):
        while not state.gameOver():
            state.makeMove(random.choice(state.getLegalMoves())+1, self.__rotateSymbol())
        
        if state.gameDraw():
            return ''
        
        return state.result
    
    def backPropagation(self, node, turn, outcome):

        reward = 0 if outcome == turn else 1

        while node is not None:
            node.N += 1
            node.Q += reward
            node = node.parent
            if outcome == '':
                reward = 0
            else:
                reward = 1 - reward

    def search(self, timeLimit):
        startTime = time.process_time()

        numRollouts = 0

        while time.process_time() - startTime < timeLimit:
            node, state = self.__selection()
            outcome = self.rollOut(state)
            self.backPropagation(node, self.symbol, outcome)
            numRollouts += 1

        self.runTime = time.process_time() - startTime
        self.numRollouts = numRollouts

    def bestMove(self):

        max_value = max(self.root.children.values(), key=lambda n: n.N).N
        max_nodes = [n for n in self.root.children.values() if n.N == max_value]
        bestChild = random.choice(max_nodes)

        return bestChild
    
    def moveRoot(self, move, symbol):
        if move in self.root.children:                                                          # Se a jogada estiver nos filhos do nó que representa a raiz:
            self.rootState.makeMove(move+1, symbol)                                             # Realizamos a jogada em questão.
            self.root = self.root.children[move]                                                # Atualizamos a raiz para descer com a jogada em questão.
            return

        if move:                                                                                # No caso de ser a primeira jogada:
            self.rootState.makeMove(move+1, symbol)                                             # Realizamos a jogada em questão.
        self.root = Node(None, None, None)                                                      # Definimos a root como um nó vazio.


    def play(self, _, move):

        self.moveRoot((move and move-1), self.playerSymbol)                                     # Definimos a root com o move do jogador.

        self.search(10)                                                                         # Fazemos uma execução do algoritmo com tempo limite de 16 segundos.
        
        mcts_move = self.bestMove().move                                                        # Determinamos a melhor jogada possível com o MCTS.
        
        self.moveRoot(mcts_move, self.iaSymbol)                                                 # Atualizamos a raiz, tendo em conta essa mesma jogada escolhida pelo MCTS.

        self.symbol = self.iaSymbol                                                             # Definimos o símbolo corretamente.
        
        return mcts_move                                                                        # Por fim, retornamos a melhor jogada possível à interface.

## **Análise MCTS**

#### **Esquema:**
<div style="text-align: center;">
    <img src="esquemas/MCTS.png" alt="Esquema MCTS" width="1200" class="center"/>
</div>

O Monte Carlo Tree Search é um algoritmo de pesquisa avançado, muito utilizado em jogos mais complexos, como por exemplo o Go e o Xadrez, devido à sua capacidade de lidar melhor com árvores de pesquisa maiores dado que este não a explora na sua totalidade, evitando assim a pesquisa tradicional que levaria rapidamente a um grande cálculo combinatório. A essencia do MCTS é escolher os nós mais promissores, após várias simulações aleatórias e o respetivo registo de vitórias para cada uma das possibilidades. O algoritmo é dividido em 4 fases, que iremos apresentar.

<br>

### **Selection**

In [ ]:
def __selection(self):
        node = self.root                                                                        # Definimos o nó que iremos manipular através da root atual.
        state = deepcopy(self.rootState)                                                        # Fazemos uma deepcopy do estado do jogo na raiz, para que não seja alterado o jogo em vigor.

        while len(node.children) != 0:                                                          # Enquanto o nó tiver filhos:
            children = node.children.values()                                                   # Obtemos os nós através do dicionário (este é do tipo dict_values()).
            max_value = max(children, key=lambda n: n.value()).value()                          # Entre esses nós, descobrimos o que tem o maior valor segundo a fórmula UCT (definida na classe Node).
            maxChildren = [n for n in children if n.value() == max_value]                       # Comparamos o maior valor com os restantes nós, porque podem existir vários com o mesmo valor.

            node = random.choice(maxChildren)                                                   # Na lista de nós com os maiores valores, escolhemos de forma aleatória um (para que seja imparcial).
            state.makeMove(node.move+1, self.__rotateSymbol())                                  # Na cópia que fizemos do jogo, fazemos a jogada correspondente ao nó que obtemos.

            if node.N == 0:                                                                     # Se o nó ainda não foi explorado:
                return node, state                                                              # Retornamos esse mesmo nó e o respetivo estado de jogo após a jogada que correspondia a esse nó.

        if self.__expand(node, state):                                                          # Se for possível fazer a expansão:
            node = random.choice(list(node.children.values()))                                  # Escolhemos aleatóriamente um dos nós.
            state.makeMove(node.move+1, self.__rotateSymbol())                                  # Fazemos a respetiva jogada, indicada por esse nó.
            
        return node, state                                                                      # No fim, retornamos o nó e o estado do jogo após a jogada.

Na *Selection*, começamos a partir da raiz R, e selecionamos nós filhos sucessivamente até atingirmos um nó folha L. A raiz é o estado atual do jogo e uma folha é qualquer nó que não tenha filhos mas que tenha um potencial filho onde nenhum/a *Rollout/Simulation* tenha sido iniciado. Quando chegarmos a esse nó folha iremos fazer a *Expansion*.

<br>

### **Expansion:**

In [ ]:
def __expand(self, parent, state):
    if state.gameOver():                                                                        # Se o jogo estiver num estado final:
        return False                                                                            # Retornamos False, indicando que não deve ser feita a expansão (dado que esta seria impossível).
        
    children = [Node(move, None, parent) for move in state.getLegalMoves()]                     # Caso contrário, criamos uma lista de nós através das possíveis jogadas disponíveis.
    parent.setChildren(children)                                                                # Atribuímos a lista de nós filhos ao respetivo nó pai.

    return True                                                                                 # Retornamos True indicando que poderá ser feita a expansão.

No caso da *Expansion*, a não ser que L (nó folha) seja um estado final (vitória/empate/derrota) para qualquer jogador, é criada uma lista com os nós com as possíveis jogadas no dado estado do jogo. Após isso é escolhido um dos nós filhos de forma aleatória. Isto porque como os nós acabaram todos de ser criados, o seu valor N (número de visitas durante as simulações) é sempre 0. Segundo a nossa implementação, o valor do nó passa a ser infinito, para que esse nó tenha prioridade sobre outros nós não explorados em futuras pesquisas. Por esse mesmo motivo é indiferente o nó que escolhemos (esta escolha poderia também ser feita por ordem númerica de forma a simplificar, mas por decisão conjunta no trabalho decidimos que seria mais correta esta abordagem para a escolha ser completamente imparcial).

Nota: No código, a escolha do nó é feita ainda na *Selection*, apesar de descrita neste mesmo contexto.

<br>

### **Rollout / Simulation:**

In [ ]:
def __rollOut(self, state):
    while not state.gameOver():                                                                 # Enquanto o jogo não estiver num estado final:
        state.makeMove(random.choice(state.getLegalMoves())+1, self.__rotateSymbol())           # Realizamos jogadas aleatórias, alternando entre os dois possíveis jogadores.
    
    if state.gameDraw():                                                                        # Se o resultado for um empate:
        return ''                                                                               # Retornamos esse mesmo indicador, que irá ser usado na BackPropagation
    
    return state.result                                                                         # Caso não seja um empate, retornamos o vencedor do jogo ('X' ou 'O')

Para os *Rollouts* ou *Simulations*, o código é bastante simples. A partir de um dado estado do jogo, fazemos jogadas alternadas entre os dois jogadores, até chegarmos a um resultado final, devolvendo no final esse mesmo resultado, que será utilizado na *BackPropagation*, para atualizar os valores dos nós. Apenas há um pormenor, quando o resultado é um empate devolvemos um estado diferente de 'X' e 'O', apenas para que na *BackPropagation* este não seja atribuído como vitória para nenhum dos jogadores.

<br>

### **BackPropagation:**

In [ ]:
def __backPropagation(self, node, turn, outcome):
                                                                                                # O turn corresponde ao turno do próximo jogador, daí a seguinte formula de cálculo:
        reward = 0 if outcome == turn else 1                                                    # Se o turno do próximo jogador for o mesmo que o resultado, a reward é 0. Caso seja diferente, a reward é 1.

        while node is not None:                                                                 # Enquanto o nó não for vazio:
            node.N += 1                                                                         # Adicionamos sempre 1 ao N, pois este representa o número de visitas ao nó.
            node.Q += reward                                                                    # Adicionamos a respetiva reward ao valor Q do nó, calculada anteriormente.
            node = node.parent                                                                  # Atualizamos o nó para o seu nó pai.
            if outcome == '':                                                                   # Necessitamos também de verificar se a outcome for um empate, onde não atribuímos pontos a nenhum jogador.
                reward = 0                                              
            else:                                                                               # Caso não seja um empate, alternamos a reward entre 1 e 0.
                reward = 1 - reward

Quanto à *BackPropagation*, esta fica encarregue de subir a árvore e atualizar os valores dos nós a partir do nó onde foi feito o *Rollout*, até à root. Como o jogo funciona por turnos, é necessário atribuir os pontos da vitória apenas quando o nó corresponde a um turno do vencedor. Atribuimos +1 por cada vitória ao valor Q do nó em questão, ou +0 caso o nó não corresponda a um turno do jogador vencedor no respectivo *Rollout*. Por fim, descartamos este *Rollout*.

Desta forma, **com estes 4 métodos, definimos a implementação do MCTS**. No entanto, definimos ainda alguns métodos para legibilidade do código assim como uma correta estruturação.

<br>

### **Search:**

In [ ]:
def search(self, timeLimit):
        startTime = time.process_time()

        numRollouts = 0

        while time.process_time() - startTime < timeLimit:
            node, state = self.__selection()
            outcome = self.rollOut(state)
            self.backPropagation(node, self.symbol, outcome)
            numRollouts += 1

        self.runTime = time.process_time() - startTime
        self.numRollouts = numRollouts

O método *Search* tem como objetivo a junção dos 4 métodos que definem o MCTS (*Selection*, *Expansion*, *Rollout*, *BackPropagation*). É de notar que o Monte Carlo Tree Search tem duas possíveis formas de ser executado:

1 -> Tempo limite<br>
2 -> Número definido de *Rollouts*

No nosso ponto de vista, é mais correto usar o tempo limite. Apesar de sabermos que dependendo da capacidade de processamento de cada computador, o número de *Rollouts* a executar pode variar e por consequência afetar a capacidade de decisão do algoritmo, também é importante a jogabilidade do mesmo. Desta forma, é possível em qualquer máquina, jogar contra o algoritmo em questão, apesar de ser factual que a performance é significativamente alterada.

Também achamos pertinente incluir uma indicação estatística para o número de Rollouts, assim como o tempo decorrido. Apesar de ser útil para questões de debug, esta também pode ter influência sobre a capacidade de decisão do algoritmo. Pois quanto mais elaborado for o código, menor quantidade de *Rollouts* conseguirá fazer, influenciando a precisão dos resultados.

<br>

### **Best Move:**

In [ ]:
def bestMove(self):

        max_value = max(self.root.children.values(), key=lambda n: n.N).N
        max_nodes = [n for n in self.root.children.values() if n.N == max_value]
        bestChild = random.choice(max_nodes)

        return bestChild

Para determinar a melhor jogada possível, usamos o método *Best Move*. O seu objetivo é apenas manipular os atributos do MCTS e devolver o melhor nó, onde queremos que o MCTS jogue.

Para isso, escolhemos o maior valor de *N*, ou seja, o nó mais explorado, onde obtivemos melhores resultados.

Após isso, comparamos com os restantes valores dos nós filhos da raiz, para obter em caso de empate, todos os nós com os melhores valores. Depois apenas fazemos uma seleção aleatória entre esses mesmos nós com o valor mais interessante, devolvendo no fim um deles.

<br>

### **Notas:**

É importante referir que na classe Node, foi utilizado a UCT (*Upper Confidence Bound Applied to Trees*). Esta fórmula tem como objetivo equilibrar a diferença entre *Exploration* e *Exploitation* quando o algoritmo tem decidir que nós procurar.

A formula é a seguinte:

$$UCT(ni)=\frac{Q}{N}+C{\sqrt\frac{log Np}{N}}$$

Com a mesma foi possível atribuir um valor bem definido a cada nó que posteriormente vem a ser usado na escolha do melhor nó.

<br>

### **Dificuldades na implementação do MCTS**

Apresentaram-se algumas dificuldades na atualização do atributo root quando este era devolvido à interface e vice-versa. Inicialmente não percebemos que esse era um dos passos do algoritmo, o que levou a uma implementação errada do mesmo. Por esse mesmo motivo, decidimos definir um atributo children (da classe Node), que é representado por um dicionário com as keys a representarem as respetivas colunas onde é feita a jogada que fazem corresponder a cada um dos nós filhos. Desta forma, ao guardar os nós filhos, com uma associação direta ao nó pai, a atualização do atributo root tornou-se mais claro e direto.

Outra das adversidades que surgiu ao longo do debug da nossa implementação foi a "exploitation constant", aplicada no cálculo do valor de cada um dos nós. Após vários testes de execução do jogo, concluímos que o valor $\sqrt{2}$ era o mais adequado para obtermos um bom equilíbrio entre a Exploration & Exploitation. Com valores maiores que este, obtinhamos frequentemente uma sobre-exploração dos nós que não nos interessavam desde o ínicio, perdendo o algoritmo dados importantes, sobre os nós relevantes para a próxima jogada.

<br>

### **Análise da relação entre Rollouts/Peças**

Relativamente ao MCTS, ainda foram analisado o número de Rollouts, relacionando estes mesmos dados com a quantidade de peças colocadas em jogo até ao momento.

Obtivemos os seguintes dados:


| Valor médio de Rollouts | Peças em Jogo |
| :---------------------: | :-----------: |
| 104250                  | 1             |
| 83000                   | 3             |
| 102250                  | 5             |
| 115000                  | 7             |
| 135600                  | 9             |
| 125000                  | 11            |
| 186000                  | 13            |
| ...                     | ...           |


<br>

Colocando estes resultados num gráfico, conseguimos tirar conclusões de uma forma mais legível:

<br>

<div>
    <img src="esquemas/grafico.png" alt="Gráfico Rollouts/Peças" width="400"/>
</div>

<br>

Verifica-se que o número de *Rollouts* aumenta significativamente com o aumento do número de peças em jogo. Isto acontece porque à medida que o jogo avança, o número de possíveis estados futuros aumenta exponencialmente, tornando necessário explorar uma maior parte da árvore de pesquisa para avaliar as consequências de cada jogada. Além disso, à medida que o jogo avança, é crucial a análise de posições estratégicas onde há uma vitória iminente. Por esse mesmo motivo, também se entende que as simulações têm de ser mais detalhadas.

Nota: O zig-zag representa uma quebra na escala.



## **Interface**

O código para uma interface que permite a escolha do algoritmo contra quem tencionamos jogar (dos apresentados acima), pode ainda ser escolhido o símbolo com que jogar.

<br>

#### **Implementação:**

In [ ]:
from searchAlgos.astar import Astar
from searchAlgos.miniMax import MinMax
from searchAlgos.mcts import MCTS
from fourGame import FourGame

# Analisa e retorna a resposta junto com o tabuleiro atual
# params: 
#
#       game | matrix (list of lists): matrix de caracteres | representa o estado atual do tabuleiro do jogo
#       result | type: int | representa o códdigo daquilo que aconteceu no jogo (-1 -> a coluna está cheia | 0 -> Caso não acha fim no jogo | 1 -> Caso de empate | 2 -> Caso de vitória)
#       winner | type: string | representa o símbolo que venceu no caso de vitória
#
# returns: type: boolean | Retorna se o jogo terminou ou não

def showResults(game, result, winner):

    print(game)  # Print do estado atual do tabuleiro

    match result:
        case -1:  # Caso quando a coluna está cheia
            print("Invalid Move! Please choose another column.")
            return False, True
        case 0:  # Caso para quando o movimento é valido mas não resulta no fim do jogo
            print("Nice Move!")
            return False, False
        case 1:  # Caso de empate
            print("It's a Draw!!")
            return True, False
        case 2:  # Caso de vitória
            print('The symbol ' + winner + ' just won!')
            return True, False

def main():
    game = FourGame(7, 6)  # Cria uma nova instância da classe FourGame
    end = False  # Inizializa end como False e usa-o para indicar que o jogo ainda não terminou

    move = input('Escolhe o símbolo com que queres jogar (X ou O): ') # Escolher com que símbolo se quer jogar 

    # Escolhe o símbolo com que o jogador quer jogar
    try:
        if move != 'X' and move != 'O':
            raise ValueError
    except ValueError:
        print('O símbolo deve ser X ou O')
        return
    
    match move:
        case 'X':
            iaSymbol = 'O'
        case 'O': 
            iaSymbol = 'X'

    # Escolhe contra que algoritmo o jogador pretende jogar
    print('===============\n| 1 -> A*     |\n| 2 -> MCTS   |\n| 3 -> MinMax |\n===============')
    algoResp = int(input('Escolhe contra que algoritmo gostarias de jogar: '))
    match algoResp:
        case 1:
            algo = Astar(iaSymbol)
        case 2:
            algo = MCTS(iaSymbol, game)
        case 3:
            algo = MinMax(iaSymbol, move)

    # Faz moves até que o jogo tenha um fim
    while not end:
        invalid = False
        while True:  # Loop para verificar o input até ser dado um movimento válido
            try:
                col = int(input('Column: '))
                if col < 1 or col > 7:
                    raise ValueError  # Raise a ValueError se o input não estiver entre 1-7
                break
            except ValueError:
                print("Invalid Input! Please enter a number from 1 to 7.")

        result, winner = game.makeMove(col, move)  # Faz um movimento na coluna col

        end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual

        if not end and not invalid:
            result, winner = game.makeMove(algo.play(game)+1, iaSymbol)  # Faz um movimento na coluna col

            end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual
        
if __name__ == '__main__':
    main()

### **Algoritmo VS Algoritmo**

Por motivos de testes e análise de dados, decidimos também fazer um código para os algoritmos jogarem entre si.

In [ ]:
from searchAlgos.astar import Astar
from searchAlgos.miniMax import MinMax
from searchAlgos.mcts import MCTS
from fourGame import FourGame

# Analisa e retorna a resposta junto com o tabuleiro atual
# params: 
#
#       game | matrix (list of lists): matrix de caracteres | representa o estado atual do tabuleiro do jogo
#       result | type: int | representa o códdigo daquilo que aconteceu no jogo (-1 -> a coluna está cheia | 0 -> Caso não acha fim no jogo | 1 -> Caso de empate | 2 -> Caso de vitória)
#       winner | type: string | representa o símbolo que venceu no caso de vitória
#
# returns: type: boolean | Retorna se o jogo terminou ou não

def showResults(game, result, winner):

    print(game)  # Print do estado atual do tabuleiro

    match result:
        case -1:  # Caso quando a coluna está cheia
            print("Invalid Move! Please choose another column.")
            return False, True
        case 0:  # Caso para quando o movimento é valido mas não resulta no fim do jogo
            print("Nice Move!")
            return False, False
        case 1:  # Caso de empate
            print("It's a Draw!!")
            return True, False
        case 2:  # Caso de vitória
            print('The symbol ' + winner + ' just won!')
            return True, False

def AstarVsMinMax():
    game = FourGame(7, 6)
    astar = Astar('O')
    minMax = MinMax('X', 'O')
    end = False

    while not end:
        result, winner = game.makeMove(astar.play(game, None)+1, 'O')  # Faz um movimento na coluna col

        end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual

        if not end and not invalid:
            result, winner = game.makeMove(minMax.play(game, None)+1, 'X')  # Faz um movimento na coluna col

            end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual

    
    
def AstarVsMTCS(): 
    game = FourGame(7, 6)
    astar = Astar('O')
    mcts = MCTS('X', game)

    end = False

    while not end:
        move = astar.play(game, None)+1
        result, winner = game.makeMove(move, 'O')  # Faz um movimento na coluna col

        end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual

        if not end and not invalid:
            result, winner = game.makeMove(mcts.play(game, move)+1, 'X')  # Faz um movimento na coluna col

            end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual


def MinMaxVsMTCS(): 
    game = FourGame(7, 6)
    minMax = MinMax('O', 'X')
    mcts = MCTS('X', game)

    end = False

    while not end:
        move = minMax.play(game, None)+1
        result, winner = game.makeMove(move, 'O')  # Faz um movimento na coluna col

        end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual

        if not end and not invalid:
            result, winner = game.makeMove(mcts.play(game, move)+1, 'X')  # Faz um movimento na coluna col

            end, invalid = showResults(game, result, winner) # Analisa e retorna a resposta junto com o tabuleiro atual


def main():

    # Escolhe que algoritmos jogam
    print('=======================\n| 1 -> A* vs MiniMax  |\n| 2 -> A* vs MCTS     |\n| 3 -> MinMax vs MCTS |\n=======================')
    algoResp = int(input('Escolhe os algoritmos para jogarem: '))
    match algoResp:
        case 1:
            AstarVsMinMax()
        case 2:
            AstarVsMTCS()
        case 3:
            MinMaxVsMTCS()
        
if __name__ == '__main__':
    main()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


#### **A\* vs MiniMax**

Neste primeiro alvo de testes, obtemos uma vitória constante do MiniMax sobre o A*. Tendo em conta que o A* se comporta
como um algoritmo Greedy, pois este não entende as jogadas do adversário e apenas joga para vencer o mais rápido possível,
acaba por ser facilmente vencido pelo MiniMax pois este é uma versão aprimorada do A*, tendo em conta as jogadas disponíveis
para o adversário até uma dada profundidadade da árvore.

<br>

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### **A\* vs MCTS**

À semelhança do frente-a-frente entre o A* e o Minimax, nos testes realizados entre estes dois algoritmos, verificamos uma
vitória irrefutável do MCTS. Facilmente este bloqueia o A* e acaba por vencer, dado o A* não ter em conta as jogadas do 
adversário, ao contrário do MCTS que é adversarial. Mesmo com um time limit baixo para o MCTS (2/4 segundos) é possível
ganhar ao A*.

<br>

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### **MiniMax vs MCTS**

Percebemos que o MCTS ganhou com mais frequência quando testamos com 8 segundos de pesquisa ou mais. Fizemos testes 
para os valores 2, 4, 6, 8 e 16 segundos. Contra o algoritmo MiniMax, cerca de 60% dos jogos foram vitórias para o 
MCTS, 10% empates e 30% vitórias para o MiniMax (amostra de 20 jogos).

<br>

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Pelos testes realizados quando colocamos os algoritmos frente-a-frente, concluímos então que os **melhores algoritmos** são:

1º -> MCTS <br>
2º -> MiniMax <br>
3º -> A*

<br>

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
